In [1]:
import sqlite3
import requests
import pandas as pd
import numpy as np
import os
from ftplib import FTP
import zipfile

In [2]:
# Connexion au serveur FTP GNU
def download_ftp_files():
    ftp = FTP("217.112.80.251")
    ftp.login(user="user_ins",passwd="123456@")  # Connexion anonyme

    # Naviguer vers le dossier qui contient un fichier zip
    ftp.cwd('/FEICOM2025/MASQUE/DATA')
    # Create Data directory if it doesn't exist
    data_dir = 'Data_Zip'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    # Liste les fichiers disponibles
    files = ftp.nlst()
    # Download all available zip files
    available_files = [f for f in files if f.endswith('.zip')]
    if available_files:
        for filename in available_files:
            # Create the full path for saving the file in Data_Zip directory
            local_filepath = os.path.join(data_dir, filename)
            # Download and save each file in Data_Zip directory
            with open(local_filepath, 'wb') as f:
                ftp.retrbinary(f'RETR {filename}', f.write)
            print(f"File '{filename}' downloaded successfully to {data_dir}/")
    else:
        print("No .zip files found")

    # Fermer la connexion FTP
    ftp.quit()


def unzip_data_file(zip_filename):
    try:
        # Create extraction directory if it doesn't exist
        extract_dir = 'extracted_data'
        if not os.path.exists(extract_dir):
            os.makedirs(extract_dir)
            
        # Unzip the file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
            print(f"Successfully extracted {zip_filename} to {extract_dir}")
            
        # Return the list of extracted files
        extracted_files = os.listdir(extract_dir)
        return extracted_files
    
    except zipfile.BadZipFile:
        print(f"Error: {zip_filename} is not a valid zip file")
        return None
    except Exception as e:
        print(f"Error extracting {zip_filename}: {str(e)}")
        return None

In [68]:
download_ftp_files()

File 'DATA1111.zip' downloaded successfully to Data_Zip/
File 'DATA1113.zip' downloaded successfully to Data_Zip/
File 'DATA1114.zip' downloaded successfully to Data_Zip/
File 'DATA1115.zip' downloaded successfully to Data_Zip/
File 'DATA1116.zip' downloaded successfully to Data_Zip/
File 'DATA1117.zip' downloaded successfully to Data_Zip/
File 'DATA1118.zip' downloaded successfully to Data_Zip/
File 'DATA1119.zip' downloaded successfully to Data_Zip/
File 'DATA1120.zip' downloaded successfully to Data_Zip/
File 'DATA1121.zip' downloaded successfully to Data_Zip/
File 'DATA1122.zip' downloaded successfully to Data_Zip/
File 'DATA1123.zip' downloaded successfully to Data_Zip/
File 'DATA1124.zip' downloaded successfully to Data_Zip/
File 'DATA1125.zip' downloaded successfully to Data_Zip/


In [10]:


def Unzip_All_Files():
    zip_dir = 'Data_Zip'
    zip_files = [f for f in os.listdir(zip_dir) if f.endswith('.zip')]
    for zip_file in zip_files:
        zip_path = os.path.join(zip_dir, zip_file)
        extracted_files = unzip_data_file(zip_path)
        if extracted_files:  # Only delete if extraction was successful
            os.remove(zip_path) 
        


In [69]:
Unzip_All_Files()

Successfully extracted Data_Zip\DATA1111.zip to extracted_data
Successfully extracted Data_Zip\DATA1113.zip to extracted_data
Successfully extracted Data_Zip\DATA1114.zip to extracted_data
Successfully extracted Data_Zip\DATA1115.zip to extracted_data
Successfully extracted Data_Zip\DATA1116.zip to extracted_data
Successfully extracted Data_Zip\DATA1117.zip to extracted_data
Successfully extracted Data_Zip\DATA1118.zip to extracted_data
Successfully extracted Data_Zip\DATA1119.zip to extracted_data
Successfully extracted Data_Zip\DATA1120.zip to extracted_data
Successfully extracted Data_Zip\DATA1121.zip to extracted_data
Successfully extracted Data_Zip\DATA1122.zip to extracted_data
Successfully extracted Data_Zip\DATA1123.zip to extracted_data
Successfully extracted Data_Zip\DATA1124.zip to extracted_data
Successfully extracted Data_Zip\DATA1125.zip to extracted_data


In [ ]:
def lire_csdb(csdb_path, table_name=None, columns=None, remove=False):
    conn = sqlite3.connect(csdb_path)
    cursor = conn.cursor()
    # Rename the table 'level-1' to 'level' if it exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='level-1';")
    if cursor.fetchone():
        cursor.execute("ALTER TABLE 'level-1' RENAME TO level;")
        conn.commit()
    # Trouver la table principale si non spécifiée
    if not table_name:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        table_name = cursor.fetchall()[0][0]  # première table
    
    # Construire la requête SQL
    if columns:
        columns_str = ", ".join(columns)
        query = f"SELECT {columns_str} FROM {table_name}"
        #query = f"SELECT * FROM {table_name}"
    else:
        query = f"SELECT * FROM {table_name}"
    
    # Lire les données
    other_file=csdb_path + ".lst"
    try:
        df = pd.read_sql_query(query, conn)
        conn.close()
        if remove==True:
            os.remove(csdb_path)
            os.remove(other_file)
        return df
    except Exception as e:
        print(f"Error reading {csdb_path}: {str(e)}")
        conn.close()
        return None
  

In [ ]:
import_col=['`level-1-id`','s03q01', 's03q02aa', 's03q02ab' , 's03q02ba', 's03q02bb', 's03q02ca']
col_time=['`level-1-id`','hd','hf']
id_col=['`level-1-id`','ms00q19','ms00q01a']
csdb_file = "extracted_data/MT1113.csdb"
df_sec = lire_csdb(csdb_path=csdb_file ,table_name='sect03',columns=import_col)
df_time=lire_csdb(csdb_path=csdb_file,table_name='mfin',columns=col_time)
df_id=lire_csdb(csdb_path=csdb_file,table_name='level',columns=id_col)

final_df=pd.merge(df_sec,df_time,on="level-1-id",how="inner")
final_df=pd.merge(final_df,df_id,on="level-1-id",how="inner")

In [29]:
df_sect04.columns

Index(['level-1-id', 's03q01'], dtype='object')

In [18]:
code=[i for i in range(1111,1126)]
import_col=['`level-1-id`', 's03q02aa', 's03q02ab', 's03q02ba', 's03q02bb', 's03q02ca']

In [ ]:
df_echantillon=pd.read_excel("ECHANTILLON.xlsx",sheet_name="AGENT")

#importer un type de donnée pour un superviseur
def extrat_eleve(superviseur=None):
    merged_df = None
    import_col=['`level-1-id`','s03q01', 's03q02aa', 's03q02ab' , 's03q02ba', 's03q02bb', 's03q02ca']
    col_time=['`level-1-id`','hd','hf']
    id_col=['`level-1-id`','ms00q19','ms00q01a']
    
    df_sup=df_echantillon[df_echantillon["SUP"]==superviseur]
    if superviseur:
        code_agent=df_sup["ENQ"].tolist()
    for agent in [i for i in range(1113,1125)]:
    #for agent in code_agent:
        #fichier="extracted_data/"+str(type) + str(agent) + ".csdb"
        fichier="extracted_data/MT" + str(agent) + ".csdb"
        try:
            if os.path.exists(fichier):
                if merged_df is None:
                    
                    df_sec = lire_csdb(csdb_path=fichier ,table_name='sect03',columns=import_col)
                    df_time=lire_csdb(csdb_path=fichier,table_name='mfin',columns=col_time)
                    df_id=lire_csdb(csdb_path=fichier,table_name='level',columns=id_col)
                    
                    final_df=pd.merge(df_sec,df_time,on="level-1-id",how="inner")
                    final_df=pd.merge(final_df,df_id,on="level-1-id",how="inner")
                                        
                    merged_df = final_df
                else:
                    
                    df_sec = lire_csdb(csdb_path=fichier ,table_name='sect03',columns=import_col)
                    df_time=lire_csdb(csdb_path=fichier,table_name='mfin',columns=col_time)
                    df_id=lire_csdb(csdb_path=fichier,table_name='level',columns=id_col)
                    
                    final_df=pd.merge(df_sec,df_time,on="level-1-id",how="inner")
                    final_df=pd.merge(final_df,df_id,on="level-1-id",how="inner")
                                        
                    temp_df = final_df
                    
                    if temp_df is not None:
                        merged_df = pd.concat([merged_df, temp_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing file {fichier}: {str(e)}")
            continue
        other_file=fichier + ".lst"
        os.remove(fichier)
        os.remove(other_file)
        merged_df["Type"]="Eleve"
        
    return merged_df

In [71]:
df_test=extrat_eleve()

In [72]:
df_test

,level-1-id,s03q01,s03q02aa,s03q02ab,s03q02ba,s03q02bb,s03q02ca,hd,hf,ms00q19,ms00q01a,Type
0,1,2,2,None,2,None,2,104128,104210,1113,1,Eleve
1,1,2,1,1,1,1,2,104348,105950,1114,1,Eleve
2,1,1,1,2,1,1,1,53322,54027,1115,14,Eleve
3,1,2,1,1,1,1,2,95017,100556,1116,1,Eleve
4,1,1,1,2,2,None,1,104504,104545,1117,1,Eleve
5,1,1,1,2,2,None,1,104049,105303,1118,5,Eleve
6,1,2,2,None,2,None,2,104314,105139,1119,2,Eleve
7,1,2,2,None,2,None,2,104234,105505,1120,1,Eleve
8,1,1,1,1,1,1,1,104436,110840,1121,9,Eleve
9,1,2,2,None,2,None,2,104309,105155,1122,12,Eleve


In [ ]:
def extrat_ecole_maire(superviseur=None):
    merged_df = None
    import_col=['`level-1-id`','ecs00q01', 'ecs00q02', 'ecs00q03','ecs00q03n' , 'ecs00q07','ecs00q08','ecs00q09','ecs00q10','ecs00q14','ecs00q14x','ecs00q14a']
    col_time=['`level-1-id`','echd','echf']
    id_col=['`level-1-id`','ecs01q03','ecs00q00']
    
    df_sup=df_echantillon[df_echantillon["SUP"]==superviseur]
    if superviseur:
        code_agent=df_sup["ENQ"].tolist()
    for agent in code_agent:
        fichier="extracted_data/ECMA" + str(agent) + ".csdb"
        try:
            if os.path.exists(fichier):
                if merged_df is None:
                    
                    df_sec = lire_csdb(csdb_path=fichier ,table_name='sect03',columns=import_col)
                    df_time=lire_csdb(csdb_path=fichier,table_name='mfin',columns=col_time)
                    df_id=lire_csdb(csdb_path=fichier,table_name='level',columns=id_col)
                    
                    final_df=pd.merge(df_sec,df_time,on="level-1-id",how="inner")
                    final_df=pd.merge(final_df,df_id,on="level-1-id",how="inner")
                                        
                    merged_df = final_df
                else:
                    
                    df_sec = lire_csdb(csdb_path=fichier ,table_name='sect03',columns=import_col)
                    df_time=lire_csdb(csdb_path=fichier,table_name='mfin',columns=col_time)
                    df_id=lire_csdb(csdb_path=fichier,table_name='level',columns=id_col)
                    
                    final_df=pd.merge(df_sec,df_time,on="level-1-id",how="inner")
                    final_df=pd.merge(final_df,df_id,on="level-1-id",how="inner")
                                        
                    temp_df = final_df
                    
                    if temp_df is not None:
                        merged_df = pd.concat([merged_df, temp_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing file {fichier}: {str(e)}")
            continue
        
        other_file=fichier + ".lst"
        os.remove(fichier)
        os.remove(other_file)
        merged_df["Type"]="Ecole-Maire"


    return merged_df

In [ ]:
df_test['type'] = 'bv'